In [2]:
from sklearn.neighbors import KernelDensity
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import geopandas
# from sklearn.cluster import DBSCAN
import pandas as pd
import torch
import numpy as np
from scipy.spatial import ConvexHull

In [3]:
from transformers import DistilBertTokenizerFast
# from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import RobertaForSequenceClassification , RobertaTokenizerFast, RobertaConfig

BASE_MODEL = 'models/2021-12-03_model-distilbert-base-uncased_loss-wiki_exploded_geonames'

TOKEN_MODEL = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizerFast.from_pretrained(TOKEN_MODEL)
# model = DistilBertForSequenceClassification.from_pretrained(BASE_MODEL)
# tokenizer = RobertaTokenizerFast.from_pretrained(TOKEN_MODEL)
# model = RobertaForSequenceClassification.from_pretrained(BASE_MODEL)
model = torch.load(BASE_MODEL, map_location=torch.device('cpu'))

In [4]:
text = '''
The author of the Book of Ezekiel presents himself as Ezekiel, the son of Buzi, born into a priestly (kohen) lineage.[2] Apart from identifying himself, the author gives a date for the first divine encounter which he presents: "in the thirtieth year".[3] Ezekiel describes his calling to be a prophet by going into great detail about his encounter with God and four "living creatures" with four wheels that stayed beside the creatures.[4] For the next five years he incessantly prophesied and acted out the destruction of Jerusalem and its temple, which was met with some opposition. However, Ezekiel and his contemporaries like Jeremiah, another prophet who was living in Jerusalem at that time, witnessed the fulfilment of their prophecies with the siege of Jerusalem by the Babylonians.
'''
len(text.split(" "))
# text = text.replace('\n', ' ').lower()
# text_split = text.split()
# n_steps = 20
# step_size = int(len(text_split)/n_steps)
# variations = [" ".join(text_split[:i]) + " " + " ".join(text_split[i+step_size:]) for i in range(n_steps)] 

127

In [5]:
tok = tokenizer(text, truncation=True, padding=True, return_tensors='pt')
with torch.no_grad():
    model.eval()
    out = model.forward(input_ids = tok['input_ids'], attention_mask=tok['attention_mask'], output_hidden_states=True)
out['logits']

tensor([[32.5108, 34.8767]])

In [6]:
last_hidden_state = out['hidden_states'][-1]
hidden_state = last_hidden_state  # (bs, seq_len, dim)
pooled_output = hidden_state[:, 0]  # (bs, dim)
rand_locs = []
with torch.no_grad():
    model.eval()
    for i in range(500):
        ones = torch.ones_like(pooled_output)
        dim1 = ones.shape[1]
        rand_dim1 = np.random.randint(0, dim1)
        ones[0, rand_dim1] = 0
        masked_output = ones * pooled_output
        pre_clf_output = model.pre_classifier(masked_output)  # (bs, dim)
        relu_output = torch.nn.ReLU()(pre_clf_output)  # (bs, dim)
        rand_loc = model.classifier(relu_output).cpu().detach().numpy().squeeze()
        rand_locs.append(rand_loc)
        
out_df = pd.DataFrame(rand_locs, columns=['lat', 'lon'])

In [7]:
kde = KernelDensity(
    kernel='gaussian',
    metric='haversine',
    bandwidth=0.0004)

In [8]:
out_df *= np.pi / 180

In [9]:
kde.fit(out_df)

KernelDensity(bandwidth=0.0004, metric='haversine')

In [10]:
conf_sample = kde.sample(1000)
conf_probas = np.exp(kde.score_samples(conf_sample))
conf_probas /= conf_probas.sum()
conf_df = pd.Series(conf_probas, name='proba').to_frame()

In [11]:
conf_sample *= 180. / np.pi
conf_df['point'] = geopandas.GeoSeries([Point(point[::-1]) for point in conf_sample])
conf_df.loc[:,['lat', 'lon']] = conf_sample

In [12]:
# lat_mean = conf_df['lat'].mean()
# lon_mean = conf_df['lon'].mean()

In [13]:
# lat_space = np.linspace(-np.pi/2 , np.pi/2, 100) 
# lon_space = np.linspace(-np.pi, np.pi, 100)
# lat_space = np.linspace(-90 , 90, 100) 
# lon_space = np.linspace(-180, 180, 100)
# X, Y = np.meshgrid(lon_space,lon_space)
# xy = np.vstack([X.ravel(), Y.ravel()]).T
# X, Y = X * 180. / np.pi, X * 180./np.pi
# Z = kde.score_samples(xy)
# Z = Z.reshape(X.shape)

In [14]:
# world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

In [15]:
# plt.figure(figsize=(16,9))
# cont = plt.contour(X, Y, Z, 30)
# world.plot(color="lightgrey")

In [16]:
conf_df['cdf'] = conf_df.sort_values('proba', ascending=False).proba.cumsum()

In [17]:
levels = 10

In [19]:
conf_df['conf_level'] = pd.qcut(conf_df.cdf, q=levels,labels=False)

In [20]:
conf_areas = []
for level in range(levels):
    print(level)
    conf_area_points = conf_df[conf_df.conf_level == level][['lat', 'lon']]
    conf_area_hull = ConvexHull(conf_area_points.values)
    vertecies = conf_area_hull.points[conf_area_hull.vertices]
    points = geopandas.GeoSeries([Point(point[::-1]) for point in vertecies])
    polygon = Polygon(points)
    conf_areas.append(polygon)

IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing


0
1
2
3
4
5
6
7
8
9


In [191]:
conf_areas

In [193]:
conf_area_series = geopandas.GeoSeries(conf_areas)

In [194]:
conf_area_series

ValueError: Null geometry supports no operations

In [202]:
import folium
from folium.plugins import HeatMap

In [209]:
maploc = folium.Map(
#     location=[conf_df.lon.mean(), conf_df.lat.mean()],
    zoom_start=11,
    tiles="Stamen Toner", 
    # min_lat=conf_df['point'].map(lambda x: x.x).min(), 
    # max_lat=conf_df['point'].map(lambda x: x.x).max(),
    # min_lon=conf_df['point'].map(lambda x: x.y).min(),
    # max_lon=conf_df['point'].map(lambda x: x.y).max())
# maploc.add_child(folium.GeoJson(
#     data=conf_area_series.to_json(), 
#     style_function=lambda x: {'fillColor': 'blue', 'stroke': False, 'fillOpacity': 0.1})
# 
)
maploc.add_child(HeatMap(out_df))
maploc

In [34]:
out_df.to_xml('test.xml')

In [33]:
pd.__version__

'1.3.4'

In [1476]:
!pip install -U pandas

In [162]:
path = geopandas.datasets.get_path('naturalearth_lowres')
df = geopandas.read_file(path)

IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: geometries must not contain null elements

IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a LinearRing
IllegalArgumentException: Shell is not a Linear

In [163]:
df.plot()

/Users/hhoferic/anaconda3/lib/python3.8/site-packages/geopandas/array.py:918: RuntimeWarning: All-NaN slice encountered
  np.nanmin(b[:, 0]),  # minx
/Users/hhoferic/anaconda3/lib/python3.8/site-packages/geopandas/array.py:919: RuntimeWarning: All-NaN slice encountered
  np.nanmin(b[:, 1]),  # miny
/Users/hhoferic/anaconda3/lib/python3.8/site-packages/geopandas/array.py:920: RuntimeWarning: All-NaN slice encountered
  np.nanmax(b[:, 2]),  # maxx
/Users/hhoferic/anaconda3/lib/python3.8/site-packages/geopandas/array.py:921: RuntimeWarning: All-NaN slice encountered
  np.nanmax(b[:, 3]),  # maxy
/Users/hhoferic/anaconda3/lib/python3.8/site-packages/geopandas/plotting.py:410: UserWarning: The GeoSeries you are attempting to plot is composed of empty geometries. Nothing has been displayed.
  warnings.warn(


<AxesSubplot:>

ValueError: cannot convert float NaN to integer

<Figure size 432x288 with 1 Axes>